In [1]:
%%bash
module list

Currently Loaded Modulefiles:
 1) texlive/2019   2) anaconda/3_5.3.1   3) gromacs2020/2020.0(default)  


In [2]:
%matplotlib inline

In [3]:
import os
import asyncio
import matplotlib.pyplot as plt
import numpy as np
import MDAnalysis as mda

In [4]:
import aimmd
import aimmd.distributed as aimmdd

# GMX engine

In [5]:
n_engines = 4

In [6]:
scratch_dir = "/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/"
#scratch_dir = "/home/think/scratch/aimmd_distributed/"
wdirs = [os.path.join(scratch_dir, f"engine_wdir{i}") for i in range(n_engines)]

In [7]:
for d in wdirs:
    if not os.path.isdir(d):
        os.mkdir(d)

## `GmxEngine` is a thin wrapper around gromacs
- it has a `prepare()` method (which will call `grompp`) and multiple methods to then rujn the simulation, namely `run()`, `run_walltime()` and `run_nsteps()`

In [8]:
engines = [aimmdd.GmxEngine(gro_file=os.path.join(scratch_dir, "gmx_infiles/conf.gro"),
                            top_file=os.path.join(scratch_dir, "gmx_infiles/topol.top"),
                            ndx_file=None,
                            #mdrun_extra_args="-nt 2",  # use this if your version of GMX is compiled with thread-MPI support
                            mdrun_extra_args="-ntomp 2",  # use this for GMX without thread-MPI support
                            )
           for _ in range(n_engines)]

In [9]:
mdps = [aimmdd.MDP(os.path.join(scratch_dir, "gmx_infiles/md.mdp"))
        for _ in range(n_engines)]

In [10]:
mdps[0].changed

False

In [11]:
for key, val in mdps[0].items():
    print(key, " : ", val)

title  :  ['test']
cpp  :  ['/lib/cpp']
include  :  ['-I../top']
define  :  []
integrator  :  ['md-vv']
dt  :  0.002
nsteps  :  -1
nstxout  :  10
nstvout  :  10
nstlog  :  10
nstenergy  :  10
nstxout-compressed  :  10
compressed-x-grps  :  ['Protein']
energygrps  :  ['Protein', 'SOL']
nstlist  :  10
ns-type  :  ['grid']
cutoff-scheme  :  ['Verlet']
rlist  :  1.1
coulombtype  :  ['PME']
rcoulomb  :  1.1
rvdw  :  1.1
tcoupl  :  ['Berendsen']
tc-grps  :  ['Protein', 'SOL']
tau-t  :  [0.1, 0.1]
ref-t  :  [300.0, 300.0]
Pcoupl  :  ['Berendsen']
tau-p  :  1.0
compressibility  :  [4.5e-05]
ref-p  :  [1.0]
gen-vel  :  ['no']
gen-temp  :  300.0
gen-seed  :  173529
constraints  :  ['all-bonds']


In [12]:
nsteps = 5
for mdp in mdps:
    mdp['nstvout'] = 5
    mdp["nstxout"] = 5
    mdp["nstxout-compressed"] = 5
    mdp["nstlist"] = 10
    mdp["continuation"] = "yes"

In [13]:
mdps[0]

{'title': ['test'], 'cpp': ['/lib/cpp'], 'include': ['-I../top'], 'define': [], 'integrator': ['md-vv'], 'dt': 0.002, 'nsteps': -1, 'nstxout': 5, 'nstvout': 5, 'nstlog': 10, 'nstenergy': 10, 'nstxout-compressed': 5, 'compressed-x-grps': ['Protein'], 'energygrps': ['Protein', 'SOL'], 'nstlist': 10, 'ns-type': ['grid'], 'cutoff-scheme': ['Verlet'], 'rlist': 1.1, 'coulombtype': ['PME'], 'rcoulomb': 1.1, 'rvdw': 1.1, 'tcoupl': ['Berendsen'], 'tc-grps': ['Protein', 'SOL'], 'tau-t': [0.1, 0.1], 'ref-t': [300.0, 300.0], 'Pcoupl': ['Berendsen'], 'tau-p': 1.0, 'compressibility': [4.5e-05], 'ref-p': [1.0], 'gen-vel': ['no'], 'gen-temp': 300.0, 'gen-seed': 173529, 'constraints': ['all-bonds'], 'continuation': ['yes']}

In [14]:
mdps[0].changed

True

In [15]:
import time

In [16]:
from state_funcs import descriptor_func

descriptor_func_wrap = aimmdd.TrajectoryFunctionWrapper(descriptor_func, {"scratch_dir": scratch_dir})

In [17]:
init_conf = aimmdd.Trajectory(trajectory_file=os.path.join(scratch_dir, "gmx_infiles/conf.trr"),
                              structure_file=os.path.join(scratch_dir, "gmx_infiles/conf.gro"),
                             )

init_conf2 = aimmdd.Trajectory(trajectory_file=os.path.join(scratch_dir, "gmx_infiles/other_conf.trr"),
                               structure_file=os.path.join(scratch_dir, "gmx_infiles/conf.gro"),
                              )

In [18]:
cv = await descriptor_func_wrap(init_conf)
print(cv)

[[-0.18253787 -1.3940568 ]]


In [19]:
cv = await descriptor_func_wrap(init_conf2)
print(cv)

[[ 3.140987  -2.4220986]]


In [20]:
start = time.time()
await asyncio.gather(*(e.prepare(starting_configuration=init_conf2, workdir=wdir, deffnm="blab", run_config=mdp)
                               for e, wdir, mdp in zip(engines, wdirs, mdps)
                               )
                            )
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 5.277810573577881 s


In [21]:
walltime = 0.005 # 0.01 h = 36 s
start = time.time()
trajs = await asyncio.gather(*(e.run_walltime(walltime) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


time elapsed: 20.323787927627563 s


In [22]:
cv = await descriptor_func_wrap(trajs[0])
cv

array([[ 3.140987 , -2.4220986],
       [ 3.0625575, -2.5032413],
       [ 3.0615392, -2.5220623],
       ...,
       [ 2.4690552, -1.244236 ],
       [ 2.580096 , -1.2232071],
       [ 2.6852977, -1.093723 ]], dtype=float32)

In [23]:
cv.shape

(4481, 2)

In [24]:
for i, e in enumerate(engines):
    print(f"engine {i} did {e.frames_done} frames (== {e.steps_done} integration steps)")

engine 0 did 4481 frames (== 22405 integration steps)
engine 1 did 4545 frames (== 22725 integration steps)
engine 2 did 4481 frames (== 22405 integration steps)
engine 3 did 4481 frames (== 22405 integration steps)


In [25]:
for t in trajs:
    print(len(t), t)

4481 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir0/blab.part0001.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir0/blab.tpr)
4545 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir1/blab.part0001.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir1/blab.tpr)
4481 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir2/blab.part0001.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir2/blab.tpr)
4481 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir3/blab.part0001.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir3/blab.tpr)


In [26]:
walltime = 0.005 # 0.01 h = 36 s
start = time.time()
trajs = await asyncio.gather(*(e.run_walltime(walltime) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")
cv = await descriptor_func_wrap(trajs[0])
cv

time elapsed: 19.324897289276123 s


/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


array([[ 2.6852977, -1.093723 ],
       [ 2.6379528, -1.0598217],
       [ 2.5858073, -1.196813 ],
       ...,
       [ 0.9575266, -2.465769 ],
       [ 0.8202443, -2.5388083],
       [ 0.7734374, -2.6414542]], dtype=float32)

In [27]:
cv.shape

(4609, 2)

In [28]:
for i, e in enumerate(engines):
    print(f"engine {i} did {e.frames_done} frames (== {e.steps_done} integration steps)")

engine 0 did 9090 frames (== 45450 integration steps)
engine 1 did 9138 frames (== 45690 integration steps)
engine 2 did 9058 frames (== 45290 integration steps)
engine 3 did 9074 frames (== 45370 integration steps)


In [29]:
for t in trajs:
    print(len(t), t)

4609 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir0/blab.part0002.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir0/blab.tpr)
4593 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir1/blab.part0002.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir1/blab.tpr)
4577 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir2/blab.part0002.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir2/blab.tpr)
4593 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir3/blab.part0002.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir3/blab.tpr)


In [30]:
start = time.time()
trajs = await asyncio.gather(*(e.run_steps(nsteps) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 0.8091819286346436 s


/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


In [31]:
cv = await descriptor_func_wrap(trajs[0])

In [32]:
cv

array([[ 0.7734374, -2.6414542],
       [ 0.8118913, -2.6223893]], dtype=float32)

In [33]:
for t in trajs:
    print(len(t), t)

2 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir0/blab.part0003.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir0/blab.tpr)
2 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir1/blab.part0003.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir1/blab.tpr)
2 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir2/blab.part0003.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir2/blab.tpr)
2 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir3/blab.part0003.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir3/blab.tpr)


In [34]:
for i, e in enumerate(engines):
    print(f"engine {i} did {e.frames_done} frames (== {e.steps_done} integration steps)")

engine 0 did 9092 frames (== 45460 integration steps)
engine 1 did 9140 frames (== 45700 integration steps)
engine 2 did 9060 frames (== 45300 integration steps)
engine 3 did 9076 frames (== 45380 integration steps)


In [35]:
u0 = mda.Universe(trajs[0].structure_file, trajs[0].trajectory_file)
ts00 = u0.trajectory[0]
print(ts00.__dict__)
ts0_1 = u0.trajectory[-1]
print(ts0_1.__dict__)

{'frame': 0, '_n_atoms': 1651, 'data': {'time': 90.87999725341797, 'step': 45440, 'lambda': 0.0, 'dt': 0.01000213623046875}, '_has_positions': True, '_has_velocities': True, '_has_forces': False, '_pos': array([[23.091908 , 18.778294 , 23.595484 ],
       [22.861214 , 18.351141 , 24.571419 ],
       [22.349789 , 18.373978 , 22.907084 ],
       ...,
       [ 3.3426323,  0.0414657,  4.06711  ],
       [ 3.8018777, 25.661602 ,  3.2306502],
       [ 2.600438 ,  0.617043 ,  3.8824751]], dtype=float32), '_unitcell': array([25.695374, 25.695374, 25.695374, 90.      , 90.      , 90.      ],
      dtype=float32), 'aux': {}, '_velocities': array([[ -4.464505  ,  -0.43065983,  -0.84200287],
       [ 19.157602  , -16.479336  ,  -2.2754023 ],
       [-19.79829   ,  -4.050318  ,  17.804329  ],
       ...,
       [  3.2672539 ,  -1.3236597 ,  -1.829297  ],
       [ 21.193462  , -16.16076   ,   9.34738   ],
       [ 24.948843  ,  20.650028  , -20.484531  ]], dtype=float32)}
{'frame': 1, '_n_atoms': 16

In [36]:
start = time.time()
trajs = await asyncio.gather(*(e.run_steps(nsteps) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 0.775115966796875 s


/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


In [37]:
cv = await descriptor_func_wrap(trajs[0])
print(cv)

[[ 0.8118913  -2.6223893 ]
 [ 0.81043917 -2.592852  ]]


In [38]:
for i, e in enumerate(engines):
    print(f"engine {i} did {e.frames_done} frames (== {e.steps_done} integration steps)")

engine 0 did 9094 frames (== 45470 integration steps)
engine 1 did 9142 frames (== 45710 integration steps)
engine 2 did 9062 frames (== 45310 integration steps)
engine 3 did 9078 frames (== 45390 integration steps)


In [39]:
for t in trajs:
    print(len(t), t)

2 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir0/blab.part0004.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir0/blab.tpr)
2 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir1/blab.part0004.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir1/blab.tpr)
2 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir2/blab.part0004.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir2/blab.tpr)
2 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir3/blab.part0004.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir3/blab.tpr)


In [40]:
u0 = mda.Universe(trajs[0].structure_file, trajs[0].trajectory_file)
ts00 = u0.trajectory[0]
print(ts00.__dict__)
ts0_1 = u0.trajectory[-1]
print(ts0_1.__dict__)

{'frame': 0, '_n_atoms': 1651, 'data': {'time': 90.88999938964844, 'step': 45445, 'lambda': 0.0, 'dt': 0.01000213623046875}, '_has_positions': True, '_has_velocities': True, '_has_forces': False, '_pos': array([[2.3049631e+01, 1.8761660e+01, 2.3596115e+01],
       [2.3041119e+01, 1.8387148e+01, 2.4619720e+01],
       [2.2160480e+01, 1.8486549e+01, 2.3028812e+01],
       ...,
       [3.3698344e+00, 2.3176935e-02, 4.0544095e+00],
       [3.9366059e+00, 2.5477859e+01, 3.3207917e+00],
       [2.8426421e+00, 7.5055647e-01, 3.7239265e+00]], dtype=float32), '_unitcell': array([25.693026, 25.693026, 25.693026, 90.      , 90.      , 90.      ],
      dtype=float32), 'aux': {}, '_velocities': array([[ -2.3491812 ,  -0.9882772 ,  -0.17966247],
       [ 16.543938  ,  16.630548  ,   6.4161544 ],
       [-12.555137  ,  12.953875  ,   9.0662    ],
       ...,
       [  2.8898945 ,  -2.2406151 ,  -0.5087193 ],
       [  3.2162476 , -20.179253  ,   5.571433  ],
       [ 18.388712  ,   4.0694695 , -11.3

In [41]:
start = time.time()
trajs = await asyncio.gather(*(e.run_walltime(0.001) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


time elapsed: 4.8293678760528564 s


In [42]:
cv = await descriptor_func_wrap(trajs[0])
print(cv)

[[ 0.81043917 -2.592852  ]
 [ 0.78017974 -2.559094  ]
 [ 0.75905216 -2.5287538 ]
 ...
 [ 2.1661081  -1.6137656 ]
 [ 2.1721003  -1.6194788 ]
 [ 2.3067749  -1.6566355 ]]


In [43]:
for i, e in enumerate(engines):
    print(f"engine {i} did {e.frames_done} frames (== {e.steps_done} integration steps)")

engine 0 did 10037 frames (== 50185 integration steps)
engine 1 did 10069 frames (== 50345 integration steps)
engine 2 did 9989 frames (== 49945 integration steps)
engine 3 did 10005 frames (== 50025 integration steps)


In [44]:
trajs

[Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir0/blab.part0005.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir0/blab.tpr),
 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir1/blab.part0005.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir1/blab.tpr),
 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir2/blab.part0005.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir2/blab.tpr),
 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir3/blab.part0005.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir3/blab.tpr)]

In [45]:
u0 = mda.Universe(trajs[0].structure_file, trajs[0].trajectory_file)
ts00 = u0.trajectory[0]
print(ts00.__dict__)
ts0_1 = u0.trajectory[-1]
print(ts0_1.__dict__)

{'frame': 0, '_n_atoms': 1651, 'data': {'time': 90.9000015258789, 'step': 45450, 'lambda': 0.0, 'dt': 0.01000213623046875}, '_has_positions': True, '_has_velocities': True, '_has_forces': False, '_pos': array([[23.041538  , 18.772116  , 23.558666  ],
       [23.178173  , 18.423756  , 24.582422  ],
       [22.059914  , 18.470963  , 23.192825  ],
       ...,
       [ 3.4087553 , 25.68991   ,  4.0551586 ],
       [ 3.8547823 , 25.27629   ,  3.3160994 ],
       [ 2.9080663 ,  0.71244365,  3.6633415 ]], dtype=float32), '_unitcell': array([25.693026, 25.693026, 25.693026, 90.      , 90.      , 90.      ],
      dtype=float32), 'aux': {}, '_velocities': array([[ -0.1494936 ,   2.0690534 ,  -7.6256304 ],
       [ 11.21241   , -12.136802  , -13.980354  ],
       [ -8.695203  ,  -8.415571  ,  23.946808  ],
       ...,
       [  4.9475284 ,  -3.0543146 ,   0.30266136],
       [-17.0493    , -17.019442  ,  -5.1568837 ],
       [ -5.127823  ,  -9.207075  ,   1.9410979 ]], dtype=float32)}
{'frame': 

In [42]:
start = time.time()
trajs = await asyncio.gather(*(e.run_walltime(0.001) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

cv = await descriptor_func_wrap(trajs[0])
print(cv)


time elapsed: 4.588096380233765 s


/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


[[ 2.5111945  -0.8283891 ]
 [ 2.5077548  -0.9544829 ]
 [ 2.4262054  -0.99842924]
 ...
 [ 1.7833954  -1.2181917 ]
 [ 1.73214    -1.1494839 ]
 [ 1.6247302  -1.1012855 ]]


In [43]:
trajs

[Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir0/blib.part0006.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir0/blib.tpr),
 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir1/blib.part0006.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir1/blib.tpr),
 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir2/blib.part0006.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir2/blib.tpr),
 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir3/blib.part0006.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir3/blib.tpr)]

In [44]:
start = time.time()
trajs = await asyncio.gather(*(e.run_steps(1) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 0.8311331272125244 s


/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


In [45]:
cv = await descriptor_func_wrap(trajs[0])
print(cv)

[[ 1.6247302 -1.1012855]]


In [46]:
start = time.time()
trajs = await asyncio.gather(*(e.run_steps(1) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")
cv = await descriptor_func_wrap(trajs[0])
print(cv)

/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


OSError: XDR read error = integer

Exception ignored in: <function ReaderBase.__del__ at 0x7f7b3f39f940>
Traceback (most recent call last):
  File "/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/coordinates/base.py", line 2152, in __del__
    self.close()
  File "/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/coordinates/XDR.py", line 179, in close
    self._xdr.close()
AttributeError: 'TRRReader' object has no attribute '_xdr'
Exception ignored in: <function ReaderBase.__del__ at 0x7f7b3f39f940>
Traceback (most recent call last):
  File "/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/coordinates/base.py", line 2152, in __del__
    self.close()
  File "/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/coordinates/XDR.py", line 179, in close
    self._xdr.close()
AttributeError: 'TRRReader' object has no attribute '_xdr'
Exception ignored in: <function ReaderBase

In [38]:
start = time.time()
trajs = await asyncio.gather(*(e.run_steps(1) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")
cv = await descriptor_func_wrap(trajs[0])
print(cv)

/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


OSError: XDR read error = integer

Exception ignored in: <function ReaderBase.__del__ at 0x7f0aefa608b0>
Traceback (most recent call last):
  File "/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/coordinates/base.py", line 2152, in __del__
    self.close()
  File "/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/coordinates/XDR.py", line 179, in close
    self._xdr.close()
AttributeError: 'TRRReader' object has no attribute '_xdr'
Exception ignored in: <function ReaderBase.__del__ at 0x7f0aefa608b0>
Traceback (most recent call last):
  File "/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/coordinates/base.py", line 2152, in __del__
    self.close()
  File "/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/coordinates/XDR.py", line 179, in close
    self._xdr.close()
AttributeError: 'TRRReader' object has no attribute '_xdr'
Exception ignored in: <function ReaderBase

In [36]:
for i, e in enumerate(engines):
    print(f"engine {i} did {e.frames_done} frames (== {e.steps_done} integration steps)")

engine 0 did 9222 frames (== 92220 integration steps)
engine 1 did 9206 frames (== 92060 integration steps)
engine 2 did 9006 frames (== 90060 integration steps)
engine 3 did 9046 frames (== 90460 integration steps)


In [29]:
start = time.time()
trajs = await asyncio.gather(*(e.run_walltime(0.01) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 36.5525848865509 s


/home/think/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


In [30]:
cv = await descriptor_func_wrap(trajs[0])
print(cv)

[[ 1.7949889 -2.3867085]
 [ 1.7086705 -2.3630643]
 [ 1.7360168 -2.3054514]
 ...
 [-1.4733312 -1.5860335]
 [-1.4640243 -1.643299 ]
 [-1.4391651 -1.8763256]]


In [31]:
for i, e in enumerate(engines):
    print(f"engine {i} did {e.frames_done} frames (== {e.steps_done} integration steps)")

engine 0 did 12460 frames (== 124600 integration steps)
engine 1 did 12700 frames (== 127000 integration steps)
engine 2 did 12494 frames (== 124940 integration steps)
engine 3 did 13316 frames (== 133160 integration steps)


In [32]:
start = time.time()
trajs = await asyncio.gather(*(e.run_walltime(0.01) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 36.527894735336304 s


/home/think/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


In [33]:
cv = await descriptor_func_wrap(trajs[0])
print(cv)

[[-1.4391651 -1.8763256]
 [-1.4037466 -1.9764206]
 [-1.4453897 -1.972756 ]
 ...
 [ 1.3600926 -2.4688985]
 [ 1.3947617 -2.3246064]
 [ 1.5066161 -2.267869 ]]


In [34]:
for i, e in enumerate(engines):
    print(f"engine {i} did {e.frames_done} frames (== {e.steps_done} integration steps)")

engine 0 did 16605 frames (== 166050 integration steps)
engine 1 did 16869 frames (== 168690 integration steps)
engine 2 did 16661 frames (== 166610 integration steps)
engine 3 did 17440 frames (== 174400 integration steps)


# propagator until state

In [13]:
import mdtraj as mdt
import numpy as np
"""
def psi(traj):
    traj = mdt.load(traj.trajectory_file, 
                    top=os.path.join(scratch_dir, "gmx_infiles/conf.gro"),  # mdt can not work with tprs, so we use theinitial gro for now
                    )
    psi = mdt.compute_psi(traj)
    return psi

def alpha_R(traj):
    traj = mdt.load(traj.trajectory_file, 
                    top=os.path.join(scratch_dir, "gmx_infiles/conf.gro"),  # mdt can not work with tprs, so we use theinitial gro for now
                    )
    psi = mdt.compute_dihedrals(traj, indices=[[6,8,14,16]])[:, 0]
    phi = mdt.compute_dihedrals(traj, indices=[[4,6,8,14]])[:, 0]
    state = np.full_like(psi, False, dtype=bool)
    # phi: -pi -> 0 
    # psi: > -50 but smaller 30 degree
    deg = 180/np.pi
    state[(phi <= 0) & (-50/deg <= psi) & (psi <= 30/deg)] = True
    return state

wrapped_alphaR = arcdd.trajectory.TrajectoryFunctionWrapper(alpha_R)
wrapped_psi = arcdd.trajectory.TrajectoryFunctionWrapper(psi)
"""

from state_funcs import alpha_R, C7_eq
wrapped_alphaR = aimmdd.TrajectoryFunctionWrapper(alpha_R, call_kwargs={"scratch_dir": scratch_dir})
wrapped_C7_eq = aimmdd.TrajectoryFunctionWrapper(C7_eq, call_kwargs={"scratch_dir": scratch_dir})

In [14]:
import aimmd.distributed.logic

In [15]:
propagators = [aimmd.distributed.logic.PropagatorUntilAnyState(states=[wrapped_alphaR, wrapped_C7_eq],
                                                               engine_cls=aimmdd.GmxEngine,
                                                               engine_kwargs={"gro_file":os.path.join(scratch_dir, "gmx_infiles/conf.gro"),
                                                                              "top_file":os.path.join(scratch_dir, "gmx_infiles/topol.top"),
                                                                               "mdrun_extra_args": "-nt 4",
                                                                              },
                                                               run_config=aimmdd.MDP(os.path.join(scratch_dir, "gmx_infiles/md.mdp")),
                                                               walltime_per_part=0.005,
                                                               
                                                              )
               for _ in range(n_engines)
               ]

Neither max_frames nor max_steps given. Setting max_frames to infinity.
Neither max_frames nor max_steps given. Setting max_frames to infinity.
Neither max_frames nor max_steps given. Setting max_frames to infinity.
Neither max_frames nor max_steps given. Setting max_frames to infinity.


In [17]:
#initial_tp_mdt = mdt.load(os.path.join(scratch_dir, "gmx_infiles/ala_400K_TP_low_barrier.h5"))

In [18]:
#initial_tp_mdt[10].save_trr(os.path.join(scratch_dir, "gmx_infiles/tp_frame.trr"))

In [16]:
starting_conf = aimmdd.Trajectory(trajectory_file=os.path.join(scratch_dir, "gmx_infiles/tp_frame.trr"),
                                  structure_file=os.path.join(scratch_dir, "gmx_infiles/tp_frame.gro"))

starting_conf = aimmdd.Trajectory(trajectory_file=os.path.join(scratch_dir, "gmx_infiles/conf.gro"),
                                  structure_file=os.path.join(scratch_dir, "gmx_infiles/conf.gro"))

In [17]:
wrapped_alphaR

TrajectoryFunctionWrapper(function=<function alpha_R at 0x7fe1fe04e700>, call_kwargs={'scratch_dir': '/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/'})

In [18]:
import mdtraj as mdt

In [19]:
mdt.load(os.path.join(scratch_dir, "gmx_infiles/conf.gro"), top=os.path.join(scratch_dir, "gmx_infiles/conf.gro"))

/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/mdtraj/formats/gro.py:307: UserWarning: WARNING: two consecutive residues with same number (ALA, ACE)
  warnings.warn("WARNING: two consecutive residues with same number (%s, %s)" % (thisresname, residue.name))
/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/mdtraj/formats/gro.py:307: UserWarning: WARNING: two consecutive residues with same number (NME, ALA)
  warnings.warn("WARNING: two consecutive residues with same number (%s, %s)" % (thisresname, residue.name))
/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/mdtraj/formats/gro.py:307: UserWarning: WARNING: two consecutive residues with same number (HOH, NME)
  warnings.warn("WARNING: two consecutive residues with same number (%s, %s)" % (thisresname, residue.name))
/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/mdtraj/core/trajectory.py:422: UserWarning: top= kwarg igno

<mdtraj.Trajectory with 1 frames, 1651 atoms, 546 residues, and unitcells at 0x7fe126c51d30>

In [20]:
mdt.load(starting_conf.trajectory_file, top=starting_conf.structure_file)

<mdtraj.Trajectory with 1 frames, 1651 atoms, 546 residues, and unitcells at 0x7fe126bff550>

In [21]:
alpha_R(starting_conf, **{"scratch_dir": scratch_dir})

/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/mdtraj/utils/validation.py:115: TypeCastPerformanceWarning: Casting unitcell_vectors dtype=float64 to <class 'numpy.float32'> 
  warnings.warn("Casting %s dtype=%s to %s " % (name, val.dtype, dtype),


array([ True])

In [22]:
await wrapped_alphaR(starting_conf)

array([ True])

In [23]:
scratch_dir

'/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/'

In [24]:
tasks = [asyncio.create_task(p.propagate_and_concatenate(starting_configuration=starting_conf,
                                                         workdir=wdir,
                                                         deffnm="blib",
                                                         tra_out=os.path.join(wdir, "blub_until_state.trr"),
                                                          ))
         for p, wdir, mdp in zip(propagators, wdirs, mdps)]

done, pending = await asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED)

In [25]:
list(done)[0]

<Task finished name='Task-5' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>

In [26]:
tasks

[<Task finished name='Task-3' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>,
 <Task finished name='Task-4' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>,
 <Task finished name='Task-5' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>,
 <Task finished name='Task-6' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() 

In [27]:
done, pending = await asyncio.wait(tasks,)# return_when=asyncio.FIRST_COMPLETED)

In [28]:
for i in done:
    print(i)
    print(tasks.index(i))

<Task finished name='Task-5' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>
2
<Task finished name='Task-6' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>
3
<Task finished name='Task-3' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>
0
<Task finished name='Task-4' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() g

In [29]:
pending

set()

In [21]:
start = time.time()
result1 = await wrapped_func(trajs[-1])
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 0.48328495025634766 s


In [22]:
start = time.time()
results = await asyncio.gather(*(wrapped_func(t) for t in trajs))
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 0.5739212036132812 s


In [23]:
trajs

[Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir0/blub.part0002.trr, topology_file=/home/think/scratch/arcd_distributed/engine_wdir0/blub.tpr),
 Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir1/blub.part0002.trr, topology_file=/home/think/scratch/arcd_distributed/engine_wdir1/blub.tpr),
 Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir2/blub.part0002.trr, topology_file=/home/think/scratch/arcd_distributed/engine_wdir2/blub.tpr),
 Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir3/blub.part0002.trr, topology_file=/home/think/scratch/arcd_distributed/engine_wdir3/blub.tpr),
 Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir4/blub.part0002.trr, topology_file=/home/think/scratch/arcd_distributed/engine_wdir4/blub.tpr),
 Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir5/blub.part0002.trr, topology_file=/home/think/scratch/arcd_dist